In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random

In [2]:
url = "https://www.popvortex.com/music/charts/top-100-songs.php"

In [3]:
response = requests.get(url)
response.status_code

200

In [4]:
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
title=[]
artist=[]


title_list=soup.select("p.title-artist cite.title")
artist_list=soup.select("p.title-artist em.artist")

num_iter = len(soup.select("p.title-artist"))

for i in range(num_iter):
    title.append(title_list[i].get_text())
    artist.append(artist_list[i].get_text())

In [6]:
hot_songs = pd.DataFrame({"title":title,
                       "artist":artist})

In [ ]:
hot_songs.head()

In [7]:
hot_songs['title'] = hot_songs['title'].str.lower()
hot_songs['artist'] = hot_songs['artist'].str.lower()

In [8]:
# the function is taken from: https://albertoviciana.medium.com/my-first-song-recommender-96994af6d6bc


def hot():
    random_song=random.choice(hot_songs['title'])
    user_input = input ('Choose a song you like:')
    
    if user_input.lower() not in list(hot_songs['title']):
        print("Sorry, I have no recommendation")
    else:
        print("If you like the song " + "'" + user_input + "'" + " then listen to this next: " + "'" + random_song + "'")        

In [ ]:
hot()

In [ ]:
hot()

In [9]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [10]:
secrets_file = open("secrets.txt","r")

In [11]:
string = secrets_file.read()

In [ ]:
string

In [12]:
string.split('\n')

['cid:f02b14fe9c4345edabf9b407b3b27756',
 'csecret:08bf95d234ad48c2abca8afb6356fdcf']

In [13]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [14]:
secrets_dict

{'cid': 'f02b14fe9c4345edabf9b407b3b27756',
 'csecret': '08bf95d234ad48c2abca8afb6356fdcf'}

In [15]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [16]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [17]:
all_tracks = get_playlist_tracks("1G8IpkZKobrIlXcVPoSIuf")
len(all_tracks)

10000

In [18]:
all_tracks[0]['track']['artists'][0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/74ASZWbe4lXaubB36ztrGX'},
 'href': 'https://api.spotify.com/v1/artists/74ASZWbe4lXaubB36ztrGX',
 'id': '74ASZWbe4lXaubB36ztrGX',
 'name': 'Bob Dylan',
 'type': 'artist',
 'uri': 'spotify:artist:74ASZWbe4lXaubB36ztrGX'}

In [19]:
all_tracks[0]['track']['artists'][0]['name']

'Bob Dylan'

In [20]:
all_tracks[0]['track']['name']

'Like a Rolling Stone'

In [21]:
all_tracks[0]['track']['uri']

'spotify:track:3AhXZa8sUQht0UEdBJgpGc'

In [22]:
all_tracks[0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [23]:
all_tracks[0]['track']['uri']

'spotify:track:3AhXZa8sUQht0UEdBJgpGc'

In [24]:
playlist_uri=[]

In [25]:
for i in range(len(all_tracks)):
    try:
        playlist_uri.append(all_tracks[i]['track']['uri'])
    except:
        playlist_uri.append('none')

In [26]:
len(playlist_uri)

10000

In [27]:
playlist_artist=[]

In [28]:
for i in range(len(all_tracks)):
    try:
        playlist_artist.append(all_tracks[i]['track']['artists'][0]['name'])
    except:
        playlist_artist.append('none')

In [29]:
len(playlist_artist)

10000

In [30]:
playlist_song_name=[]

In [31]:
for i in range(len(all_tracks)):
    try:
        playlist_song_name.append(all_tracks[i]['track']['name'])
    except:
        playlist_song_name.append('none')

In [32]:
len(playlist_song_name)

10000

In [33]:
playlist=pd.DataFrame(
    {'song_name': playlist_song_name,
     'artists': playlist_artist,
     'uri': playlist_uri
    })

In [34]:
playlist.song_name.value_counts()

Summertime                                          6
Crazy                                               4
Fire                                                4
Girlfriend                                          4
Runaway                                             3
                                                   ..
Chinese Rocks - L.A.M.F. - the Track LP restored    1
Rez                                                 1
Keep On Running - Mono Version                      1
ghostin                                             1
Come Dancing                                        1
Name: song_name, Length: 9689, dtype: int64

In [ ]:
playlist.artists.value_counts()

In [ ]:
playlist['uri'].isna().sum()

In [ ]:
playlist.isna().sum()

In [ ]:
playlist['uri'][0]

In [ ]:
song_uri = playlist['uri'][0]

In [ ]:
sp.audio_features(song_uri)

In [ ]:
# audio_features = []
# counter = 0  # setting a counter so that we can stop the code for 10 seconds when counter is 100

# for item in playlist_uri:
#     features = sp.audio_features(item)
#     audio_features.append(features)

#     counter = counter + 1
    
#     if counter % 100 == 0:
#         sleep(10)

In [ ]:
[None] in audio_features # some of the audiofeatures have [None] because there was no trackuri

In [ ]:
danceability = []
energy = []
loudness = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []

In [ ]:
for i in range(len(audio_features)):
    #making sure we only run the for loop when it is not [None]
    if audio_features[i] != [None]:  
        danceability.append(audio_features[i][0]['danceability'])
        energy.append(audio_features[i][0]['energy'])
        loudness.append(audio_features[i][0]['loudness'])
        speechiness.append(audio_features[i][0]['speechiness'])
        acousticness.append(audio_features[i][0]['acousticness'])
        instrumentalness.append(audio_features[i][0]['instrumentalness'])
        liveness.append(audio_features[i][0]['liveness'])
        valence.append(audio_features[i][0]['valence'])
        tempo.append(audio_features[i][0]['tempo'])

In [ ]:
features = pd.DataFrame({'danceability' : danceability,
                          'energy': energy,
                          'loudness': loudness,
                          'speechiness': speechiness,
                           'acousticness': acousticness,
                           'instrumentalness': instrumentalness,
                           'liveness': liveness,
                           'valence': valence,
                           'tempo': tempo
                          })

In [ ]:
features

In [ ]:
spotify = pd.concat([playlist, features], axis=1)

In [ ]:
spotify.head()

In [ ]:
spotify.to_csv('spotify_final.csv')

In [ ]:
features_df = pd.DataFrame(features)
features_df.head()
features.shape

In [ ]:
spotify.describe().T   
#we need to scale in order for all features to have the same weight

In [ ]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(features_df)

In [ ]:
pd.DataFrame(X,columns=features_df.columns).head()

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
K = range(3, 10)
inertia = []


for k in K:
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method showing the optimal k')

In [ ]:
from sklearn.metrics import silhouette_score
K = range(3, 10)
silhouette = []

for k in K:
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    silhouette.append(silhouette_score(X, kmeans.predict(X)))


plt.figure(figsize=(16,8))
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Slhouette score showing the optimal k')

In [ ]:
# optimal k looks like 6

In [ ]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(X)

clusters = kmeans.predict(X)

# Check the size of the clusters
pd.Series(clusters).value_counts().sort_index()

In [ ]:
#loop for plotting first(0) column with all the rest starting from energy
for i in range(1,len(features_df.columns)):    
    plt.figure(figsize=(5, 5))
    plt.scatter(X[:, 0], X[:, i], c=clusters,alpha=0.5)
    plt.title(f'Danceability vs {features_df.columns[i]}')
    plt.show()
#from Asha

In [ ]:
data = pd.read_csv('spotify_final.csv')
# data.head()

In [ ]:
data.shape

In [ ]:
data = data.drop('Unnamed: 0', axis=1)
data.head()

In [ ]:
data['cluster'] = clusters
data.head()

In [ ]:
clusters.sum()